In [1]:
!python3 -m pip install --force-reinstall --no-cache -r requirements.txt 


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3.13 -m pip install --upgrade pip
error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try brew install
    xyz, where xyz is the package you are trying to
    install.
    
    If you wish to install a Python library that isn't in Homebrew,
    use a virtual environment:
    
    python3 -m venv path/to/venv
    source path/to/venv/bin/activate
    python3 -m pip install xyz
    
    If you wish to install a Python application that isn't in Homebrew,
    it may be easiest to use 'pipx install xyz', which will manage a
    virtual environment for you. You can install pipx with
    
    brew install pipx
    
    You may restore the old behavior of pip by passing
    the '--break-system-packages' flag to pip, or by adding
    'break-system-packages = true' to your pip.conf file. The latter
    will permanently disable this erro

In [2]:
!ls

14-create-agent-with-clear-box-orchestration.ipynb
README.md
boto3
custom_orchestration_example(1).ipynb
images
lambda_react_streaming_claude.js
lambda_rewoo_mistral.py
requirements.txt


In [3]:
import boto3
import json
import time
import sys
import os
from agents import create_agent, invoke_agent_helper
from knowledge_bases import KnowledgeBasesForAmazonBedrock

ModuleNotFoundError: No module named 'agents'

In [ ]:
agent_name = 'restaurant-assistant'
s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
knowledge_base_name = f'{agent_name}-kb'
suffix = f"{region}-{account_id}"
knowledge_base_description = "Knowledge Base containing the restaurant menu's collection"
bucket_name = f'{agent_name}-{suffix}'

In [ ]:
kb = KnowledgeBasesForAmazonBedrock()
kb_id, ds_id = kb.create_or_retrieve_knowledge_base(
    knowledge_base_name,
    knowledge_base_description,
    bucket_name
)

In [ ]:
def upload_directory(path, bucket_name):
        for root,dirs,files in os.walk(path):
            for file in files:
                file_to_upload = os.path.join(root,file)
                print(f"uploading file {file_to_upload} to {bucket_name}")
                s3_client.upload_file(file_to_upload,bucket_name,file)

In [ ]:
upload_directory("kb_docs", bucket_name)

# sync knowledge base
kb.synchronize_data(kb_id, ds_id)

## Defining agent configuration

Let's now define the configuration for our restaurant assistant. Let's provide some data of what to do in case of situations where the agent cannot answer the user query and some more generic information about the restaurant

```
You are a restaurant assistant helping ‘The Regrettable Experience’ handle reservations. You can talk about the menus, create new bookings, get the details of an existing booking or delete an existing reservation. You reply always politely and mention the name of the restaurant in the reply. NEVER skip the name of the restaurant in the start of a new conversation. If customers ask about anything that you cannot reply, please provide the following phone number for a more personalized experience: +1 999 999 99 9999.

Some information that will be useful to answer your customer's questions:
The Regrettable Experience Address: 101W 87th Street, 100024, New York, New York
Opening hours: 
- Mondays - Fridays: 11am - 2pm and 5pm - 10pm
- Saturdays: 11am - 11pm
- Sundays: 11am - 8pm
```

For this agent, we will also use `Claude 3 Haiku` model in order to provide faster answers to the user.

For the action group we will provide 3 functions:
* `get_booking_details` to retrieve the details of an existing booking
* `create_booking` to create a new restaurant reservation and
* `delete_booking` to delete an existing reservation

Finally, we also provide some knowledge base configuration including a `kb_instruction` of when to use this knowledge base

In [ ]:
agent_foundation_model = "anthropic.claude-3-5-sonnet-20240620-v1:0"
agent_instruction = """You are a restaurant assistant helping ‘The Regrettable Experience’ handle reservations. You can talk about the menus, create new bookings, get the details of an existing booking or delete an existing reservation. You reply always politely and mention the name of the restaurant in the reply. NEVER skip the name of the restaurant in the start of a new conversation. If customers ask about anything that you cannot reply, please provide the following phone number for a more personalized experience: +1 999 999 99 9999.

Some information that will be useful to answer your customer's questions:
The Regrettable Experience Address: 101W 87th Street, 100024, New York, New York
Opening hours: 
- Mondays - Fridays: 11am - 2pm and 5pm - 10pm
- Saturdays: 11am - 11pm
- Sundays: 11am - 8pm"""
agent_description = "Agent in charge of a restaurants table bookings"

functions = [
    {
        'name': 'get_booking_details',
        'description': 'Retrieve details of a restaurant booking',
        'parameters': {
            "booking_id": {
                "description": "The ID of the booking to retrieve",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        'name': 'create_booking',
        'description': 'Create a new restaurant booking',
        'parameters': {
            "date": {
                "description": "The date of the booking in the format YYYY-MM-DD",
                "required": True,
                "type": "string"
            },
            "name": {
                "description": "Name to idenfity your reservation",
                "required": True,
                "type": "string"
            },
            "hour": {
                "description": "The hour of the booking in the format HH:MM",
                "required": True,
                "type": "string"
            },
            "num_guests": {
                "description": "The number of guests for the booking",
                "required": True,
                "type": "integer"
            }
        }
    },
    {
        'name': 'delete_booking',
        'description': 'Delete an existing restaurant booking',
        'parameters': {
            "booking_id": {
                "description": "The ID of the booking to delete",
                "required": True,
                "type": "string"
            }
        }
    },
]

action_group_config = {
    'name': 'TableBookingsActionGroup',
    'description': 'Actions for getting table booking information, create a new booking or delete an existing booking',
    'functions': functions,
    'lambda_function_name': f'{agent_name}-lambda',
    'lambda_file_path': 'lambda_function.py',
    'environment': {
        'Variables': {
            'booking_table_name': f'{agent_name}-table'
        }
    },
    'dynamodb_table_name': f'{agent_name}-table',
    'dynamodb_attribute_name': 'booking_id'
}

kb_config = {
    'kb_id': kb_id,
    'kb_instruction': 'Access the knowledge base when customers ask about the plates in the menu.'
}

In [ ]:
ra_react_agent_id, ra_react_agent_alias_id, ra_react_agent_alias_arn = create_agent(
    agent_name,
    agent_instruction,
    agent_foundation_model=agent_foundation_model,
    agent_description=agent_description,
    action_group_config=action_group_config,
    kb_config=kb_config,
    create_alias=False
)

In [ ]:
ra_react_agent_id, ra_react_agent_alias_id, ra_react_agent_alias_arn

In [ ]:
from datetime import datetime
today = datetime.today().strftime('%b-%d-%Y')
today

In [ ]:
%%time
import uuid
session_id:str = str(uuid.uuid1())

query = "Hi, I want to reserve a table for 4 people, at 9pm tonight."
session_state={
    "promptSessionAttributes": { 
         "Customer Name" : "Maria",
         "Today": today
      },
}
response = invoke_agent_helper(
    query, session_id, ra_react_agent_id, ra_react_agent_alias_id, enable_trace=True, session_state=session_state
)
print(response)

In [ ]:
%%time
import uuid
session_id:str = str(uuid.uuid1())
query = """What is in the kids menu?"""
response = invoke_agent_helper(
    query, session_id, ra_react_agent_id, ra_react_agent_alias_id, enable_trace=True
)
print(response)

In [ ]:
agent_name = 'restaurant-assistant-rewoo2'
custom_orchestration_lambda = {
    'lambda_function_name': 'rewoo-lambda',
    'lambda_file_path': 'lambda_rewoo.py'
}
action_group_config = {
    'name': 'TableBookingsActionGroup',
    'description': 'Actions for getting table booking information, create a new booking or delete an existing booking',
    'functions': functions,
    'lambda_function_name': f'{agent_name}-lambda',
    'lambda_file_path': 'lambda_function.py',
    'environment': {
        'Variables': {
            'booking_table_name': f'{agent_name}-table'
        }
    },
    'dynamodb_table_name': f'{agent_name}-table',
    'dynamodb_attribute_name': 'booking_id'
}
ra_rewoo_agent_id, rra_rewoo_agent_alias_id, ra_rewoo_agent_alias_arn = create_agent(
    agent_name,
    agent_instruction,
    agent_foundation_model=agent_foundation_model,
    agent_description=agent_description,
    action_group_config=action_group_config,
    kb_config=kb_config,
    custom_orchestration_lambda=custom_orchestration_lambda,
    create_alias=False
)

In [ ]:
ra_rewoo_agent_id, rra_rewoo_agent_alias_id, ra_rewoo_agent_alias_arn

In [ ]:
%%time
import uuid
session_id:str = str(uuid.uuid1())
query = "Hi, I want to reserve a table for 4 people, at 9pm tonight (November 20th 2024). In the name of Maria"
session_state={
    'sessionAttributes': {
        'lambda': 'arn:aws:lambda:us-east-1:351940157371:function:rewoo-lambda'
    }
}
response = invoke_agent_helper(
    query, session_id, ra_rewoo_agent_id, rra_rewoo_agent_alias_id, enable_trace=True, session_state=session_state
)
print(response)

In [ ]:
%%time
import uuid
session_id:str = str(uuid.uuid1())
query = "what is in the kids menu?"
response = invoke_agent_helper(
    query, session_id, ra_rewoo_agent_id, rra_rewoo_agent_alias_id, enable_trace=True, session_state=session_state
)
print(response)